#### load libraries and define db_connection 

In [ ]:
import os
import json
import time
from datetime import datetime, timezone
import psycopg2
from psycopg2.extras import execute_values
import aiosql
import multiprocessing as mp
import pandas as pd
import altair as alt
from dotenv import load_dotenv

# import environment variables
load_dotenv(override=True)

# import sql from folder
queries = aiosql.from_path("./sql", psycopg2)

# prepare db connection
user = os.getenv('USER')
pwd  = os.getenv('PASS')
host = os.getenv('HOST')
port = os.getenv('PORT')
db   = os.getenv('DB')
connect_db = f"postgresql://{user}:{pwd}@{host}:{port}/{db}"

#### tear down and set stuff up

In [ ]:
with psycopg2.connect(connect_db) as conn:
    try:
        # drop assets
        queries.ddl.drop_table_vertex_2_edge(conn)
        queries.ddl.drop_table_junctioned_edges(conn)
        queries.ddl.drop_type_segments_processing(conn)
        queries.ddl.drop_table_segments(conn)
        queries.ddl.drop_type_edge_processing(conn)
        #queries.ddl.drop_table_log(conn)
        
        # rebuild assets
        queries.ddl.create_table_vertex_2_edge(conn)
        queries.ddl.create_table_junctioned_edges(conn)
        queries.ddl.create_table_segments(conn)
        queries.ddl.create_type_segments_processing(conn)
        queries.ddl.create_type_edge_processing(conn)
        #queries.ddl.create_table_log(conn)
        
        # create and replace assets
        queries.dml.create_procedure_join_vertex_2_edge(conn)
        queries.dml.create_procedure_process_junctions_and_edges(conn)
        
        conn.commit()
    
    except psycopg2.Error as e:
        print(e)

#### download sources and copy data to DB

#### preprocess data in the DB

##### *define global variables for the run*

In [ ]:
# define the processing variables

# run start
# execute workstep 
run_start_date = datetime.now(timezone.utc).isoformat()

# chunk_size  = the batch size for each partial process
chunk_size = 100000

# reduce chunk_size for the enhancement and segmentation process
edge_processing_chunk_size = int(chunk_size / 1)

# concurrency = the number of parallel processes 
concurrency = 6

# run_id
run_id = int(time.time())

##### *define the wrapper functions for multiprocessing*

###### *joining the vertices to the closest input edge*

In [ ]:
def call_join_vertex_2_edge(lower_bound: int, upper_bound: int, chunk_size: int, run_id: int):
    """
    Call the junctioning procedure that 
    joins every POI to its closest edge
    """
    params = (lower_bound, upper_bound, chunk_size, run_id)
    with psycopg2.connect(connect_db) as conn:
        with conn.cursor() as cur:
            cur.execute(queries.dml.join_vertex_2_edge.sql, params)
        conn.commit()

def parallel_call_join_vertex_2_edge(params_list, concurrency):
    """
    Execute the procedure for each
    chunk in parallel
    """
    with mp.Pool(processes=concurrency) as pool:
        pool.starmap(call_join_vertex_2_edge,params_list)
    

###### *enhancing and segmentizing the input edges*

In [ ]:
def call_process_junctions_and_edges(lower_bound: int, upper_bound: int, chunk_size: int, run_id: int):
    """
    Call the junctioning procedure that 
    joins every POI to its closest edge
    """
    params = (lower_bound, upper_bound, chunk_size, run_id)
    with psycopg2.connect(connect_db) as conn:
        with conn.cursor() as cur:
            cur.execute(queries.dml.process_junctions_and_edges.sql, params)
        conn.commit()

def parallel_call_process_junctions_and_edges(params_list, concurrency):
    """
    Execute the procedure for each
    chunk in parallel
    """
    with mp.Pool(processes=concurrency) as pool:
        pool.starmap(call_process_junctions_and_edges,params_list)

##### *execute processing steps*

###### *parallel step: joining vertices to closest edge*

In [ ]:
# identify the list of lower bound IDs to
# select the chunks for parallel processing
# for the workstep "join_vertices_2_edge"

work_step = 'identify_chunks_for_join_vertices_2_edge'
start_date = datetime.now(timezone.utc).isoformat()

with psycopg2.connect(connect_db) as conn:
    bounds = list(queries.dml.find_bounds_in_poi_table(conn, chunk_size=chunk_size))

bounds_list = []
for row in bounds:
    bound = row[0]
    bounds_list.append(bound)

params_list = [(bounds_list[i], bounds_list[i+1], chunk_size, run_id) for i in range(len(bounds_list)-1)]
i = len(bounds_list)-1
params_list.append((bounds_list[i],bounds_list[i]+1, chunk_size, run_id))
params_list=params_list[:36]
#params_list

end_date = datetime.now(timezone.utc).isoformat()
message = '{"idx":1}'
log_level = "INFO"
with psycopg2.connect(connect_db) as conn:
    queries.dml.write_to_log(conn,log_level=log_level,run_id=run_id,start_date=start_date,end_date=end_date,work_step=work_step,chunk_size=chunk_size,message=message)
    conn.commit() 

In [ ]:
# execute workstep 
work_step = 'join_vertex_2_edge'
start_date = datetime.now(timezone.utc).isoformat()
parallel_call_join_vertex_2_edge(params_list,concurrency)
end_date = datetime.now(timezone.utc).isoformat()
message = '{"idx":2}'
log_level = "INFO"
with psycopg2.connect(connect_db) as conn:
    queries.dml.write_to_log(conn,log_level=log_level,run_id=run_id,start_date=start_date,end_date=end_date,work_step=work_step,chunk_size=chunk_size,message=message)
    conn.commit()  

###### *serial step: index vertex_2_edge on edge_id for further processing*

In [ ]:
# create index on vertex_2_edge (id)
work_step = 'create_index_on_vertex_2_edge'
start_date = datetime.now(timezone.utc).isoformat()
with psycopg2.connect(connect_db) as conn:
    queries.ddl.create_index_vertex_2_edge_edge_id_idx(conn)
    conn.commit()   
end_date = datetime.now(timezone.utc).isoformat()
message = '{"idx":3}'
log_level = "INFO"
with psycopg2.connect(connect_db) as conn:
    queries.dml.write_to_log(conn,log_level=log_level,run_id=run_id,start_date=start_date,end_date=end_date,work_step=work_step,chunk_size=chunk_size,message=message)
    conn.commit()    

###### *parallel step: enhance and segmentize the input edges* 

In [ ]:
# identify the list of lower bound IDs to
# select the chunks for parallel processing
# for the work_step "process_junctions_end_edges"

work_step = 'identify_chunks_for_process_junctions_end_edges'
start_date = datetime.now(timezone.utc).isoformat()


with psycopg2.connect(connect_db) as conn:
    bounds = list(queries.dml.find_bounds_in_vertex_2_edge(conn, chunk_size=edge_processing_chunk_size))

bounds_list = []
for row in bounds:
    bound = row[0]
    bounds_list.append(bound)

params_list = [(bounds_list[i], bounds_list[i+1], edge_processing_chunk_size, run_id) for i in range(len(bounds_list)-1)]
i = len(bounds_list)-1
params_list.append((bounds_list[i],bounds_list[i]+1, edge_processing_chunk_size, run_id))
#params_list=params_list[:10]
#params_list

end_date = datetime.now(timezone.utc).isoformat()
message = '{"idx":4}'
log_level = "INFO"
with psycopg2.connect(connect_db) as conn:
    queries.dml.write_to_log(conn,log_level=log_level,run_id=run_id,start_date=start_date,end_date=end_date,work_step=work_step,chunk_size=chunk_size,message=message)
    conn.commit() 

In [ ]:
# execute workstep "process_junctions_and_edges"
work_step = 'process_junctions_and_edges'
start_date = datetime.now(timezone.utc).isoformat()
parallel_call_process_junctions_and_edges(params_list,concurrency)
end_date = datetime.now(timezone.utc).isoformat()
message = '{"idx":5}'
log_level = "INFO"
with psycopg2.connect(connect_db) as conn:
    queries.dml.write_to_log(conn,log_level=log_level,run_id=run_id,start_date=start_date,end_date=end_date,work_step=work_step,chunk_size=chunk_size,message=message)
    conn.commit() 

###### *serial step: index segments on geometry for further processing*

In [ ]:
# create geometry index on segments
work_step = 'create_geometry_index_on_segments'
start_date = datetime.now(timezone.utc).isoformat()  
with psycopg2.connect(connect_db) as conn:
    queries.ddl.create_index_segments_geom_idx(conn)
    conn.commit() 
end_date = datetime.now(timezone.utc).isoformat()
message = '{"idx":6}'
log_level = "INFO"
with psycopg2.connect(connect_db) as conn:
    queries.dml.write_to_log(conn,log_level=log_level,run_id=run_id,start_date=start_date,end_date=end_date,work_step=work_step,chunk_size=chunk_size,message=message)
    conn.commit()         

In [ ]:
# run log entry

# run end
work_step = 'total_run'
start_date = run_start_date
end_date = datetime.now(timezone.utc).isoformat()
message = {"idx":0,
           "concurrency": concurrency,
           "chunk_size": chunk_size,
           "edge_processing_chunk_size": edge_processing_chunk_size
           }
message = json.dumps(message)
log_level = "INFO"
with psycopg2.connect(connect_db) as conn:
    queries.dml.write_to_log(conn,log_level=log_level,run_id=run_id,start_date=start_date,end_date=end_date,work_step=work_step,chunk_size=chunk_size,message=message)
    conn.commit()  

In [ ]:
# get data for performance display
with psycopg2.connect(connect_db) as conn:
    with queries.dml.avg_workstep_duration_item_count_cursor(conn) as cur:
        rows = cur.fetchall()
        col_names = [desc[0] for desc in cur.description]
df=pd.DataFrame(rows, columns=col_names)

# Create a scatter plot
chart = (
    alt.Chart(df)
    .mark_point()
    .encode(
        x=alt.X('avg_item_count').scale(type="log"),
        y=alt.Y('avg_duration').scale(type="linear"),
        color='concurrency',
        shape='work_step',
        tooltip=[]
    )
    .properties(
        width=800,
        height=300,
        title='Scatter Plot of item_count vs. duration'
    ).interactive()
)

jchart = alt.JupyterChart(chart)
jchart
